In [1]:

from fastapi import HTTPException
from typing import Any, Dict, Optional
import json
import hmac
import hashlib
import httpx

AGENT_SECRET = 'dEcartes2026'

def json_dumps(payload: dict) -> bytes:
    return json.dumps(payload, separators=(",", ":"), sort_keys=True).encode()


def call_agent_hmac(ip, path, secret=AGENT_SECRET, json_body=None, method="POST", timeout=5):
    url = f"http://{ip}:8000{path}"
    
    if method == "DELETE":
        # Use path param (fqdn) as body for HMAC
        fqdn = path.split("/")[-1]
        body_bytes = fqdn.encode()
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers = {
            "Content-Type": "text/plain",
            "X-Signature": signature,
        }
        res = httpx.request("DELETE", url, content=body_bytes, headers=headers, timeout=timeout)

    else:
        headers = {"Content-Type": "application/json"}
        body_bytes = json_dumps(json_body) if json_body else b"{}"
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers["X-Signature"] = signature
        
        res = httpx.post(url, content=body_bytes, headers=headers, timeout=timeout)

    try:
        res.raise_for_status()
    except httpx.HTTPStatusError as e:
        raise HTTPException(
            status_code=e.response.status_code,
            detail=f"Agent error: {e.response.text}"
        )
    return res


In [2]:
# Load JSON from file
with open("../json/dnsproof.org.json", "r") as f:
    payload = json.load(f)

# Send request
response = call_agent_hmac(
    ip="136.114.103.240",
    path="/internal/dns/push",
    json_body=payload
)

print(response.json())

{'status': 'success'}


#### DNSSEC

In [6]:
# Send request
response = call_agent_hmac(
    ip="136.114.103.240",
    path="/internal/dns/dnssec/sign/dnsproof.org"
)

print(response.json())

{'status': 'signed', 'created_keys': False, 'rrsig_records': ['SOA 8 2 3600 20260208092018 20260111092018 17775 @ N6BlfUWcJFVcT9yE7H4RsaQsppkD+4va LevI7kCuhwLWXk2eUBzrnwkoZ6MGU247 TfYpQohLQMweFD0bg6+r9QM4iwyD+otl Y5j37eAHaSkTf60l6yJ86r/Kn609ii/B OGYgkJCIPz48YGUjgmZh9FcZw/JnNHjf L231ZjkU2eI=', 'NS 8 2 3600 20260208092018 20260111092018 17775 @ RY1LptjZTvzoXpQO0w2OA4Yy2hmC2mlx DzCaYNnKAkiuEsrCcaJA2I0UYO7tpLhw ZB4B5y6DqFoXpObfC6QXPy/uGrUGmz0T pYqW/a/vxq0YGLT42uhM0Q/fhJXecX7h 4SwS7XhLLuM8VPuBJnFYDOWc87NkXLmW tLtkb0uYw+A=', 'MX 8 2 3600 20260208092018 20260111092018 17775 @ fK914Tu84bvz9y5Ni1BmiFAK1a4zcSq1 MyNk6ut74iW16+99/QY8aVzIjDQf3lZe PpaLXfotRwNcRW8rIiDTibqXOGTCZhJp WIjO/LgQ7OF8DOM1TFVAIVSmrwRCWKuJ 1rJgCfn4ISiux4qr0xKYbAcuk70NsRao lnbI1rKbXxU=', 'TXT 8 2 3600 20260208092018 20260111092018 17775 @ oiMLgLXjW9Yncd80Ngu1ga/zHv8F3Gb6 ducD7V+mjgDWA09K3icUAVtCA9qPVBFH TT+u6XlWfSEzzL5k7nkAn9FVb9YgI0hK 5cBZurozHoXmB5krUD5nsnr7zQMXczwA 36tZnMjPsnngxKep92vve6WyNt8IMoGM +Mil8hYWkZk=', 'DNSKEY 8 2 